In [1]:
from numba import jit
import numpy as np

# Propensity calculation

In [2]:
def calculate_propensities(y, k, rate_involvement_matrix):
    # product along column in rate involvement matrix
    # with states raised to power of involvement
    # multiplied by rate constants == propensity
    # dimension of y is expanded to make it a column vector
    return np.prod(np.expand_dims(y, axis=1)**rate_involvement_matrix, axis=0) * k

In [3]:
@jit(nopython=True)
def jit_calculate_propensities(y, k, rate_involvement_matrix):
    # product along column in rate involvement matrix
    # with states raised to power of involvement
    # multiplied by rate constants == propensity
    # dimension of y is expanded to make it a column vector
    intensity_power = np.expand_dims(y, axis=1)**rate_involvement_matrix
    product_down_columns = np.ones(len(k))
    for i in range(0, len(y)):
        product_down_columns = product_down_columns * intensity_power[i]
    return product_down_columns * k

In [4]:
# a realistically sparse matrix of rate involvements
n_species = 10
n_pathways = 40
poisson_involvement = np.random.poisson(0.3, (n_species,n_pathways))
y = np.random.random(n_species)
k = np.random.random(n_pathways)

In [5]:
assert(np.allclose(calculate_propensities(y,k,poisson_involvement), jit_calculate_propensities(y, k, poisson_involvement)))

In [6]:
%%timeit
calculate_propensities(y, k, poisson_involvement)

6.4 µs ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
%%timeit
jit_calculate_propensities(y, k, poisson_involvement)

KeyboardInterrupt: 

# dydt

In [ ]:
def dydt(t, y_expanded, k_of_t, N, rate_involvement_matrix, deterministic_mask, stochastic_mask, hitting_point):
    # by fiat the last entry of y will carry the integral of stochastic rates
    y = y_expanded[:-1]
    #print("y at start of dydt", y)
    propensities = calculate_propensities(y, k_of_t(t), rate_involvement_matrix)
    deterministic_propensities = propensities * deterministic_mask
    stochastic_propensities = propensities * stochastic_mask

    dydt = np.zeros_like(y_expanded)
    # each propensity feeds back into the stoich matrix to determine
    # overall rate of change in the state
    # https://en.wikipedia.org/wiki/Biochemical_systems_equation
    dydt[:-1] = N @ deterministic_propensities
    dydt[-1]  = sum(stochastic_propensities)
    #print("t", t, "y_expanded", y_expanded, "dydt", dydt)
    return dydt

def jit_dydt(t, y_expanded, k_of_t, N, rate_involvement_matrix, deterministic_mask, stochastic_mask, hitting_point):
    # by fiat the last entry of y will carry the integral of stochastic rates
    y = y_expanded[:-1]

    propensities = jit_calculate_propensities(y, k_of_t(t), rate_involvement_matrix)
    rates, sum_stochastic = _jit_dydt(y, N.astype(float), propensities, deterministic_mask, stochastic_mask)
    dydt = np.zeros_like(y_expanded)
    dydt[:-1] = rates
    dydt[-1]  = sum_stochastic
    #print("t", t, "y_expanded", y_expanded, "dydt", dydt)
    return dydt

@jit(nopython=True)
def _jit_dydt(y, N, propensities, deterministic_mask, stochastic_mask):
    deterministic_propensities = propensities * deterministic_mask
    stochastic_propensities = propensities * stochastic_mask

    # each propensity feeds back into the stoich matrix to determine
    # overall rate of change in the state
    # https://en.wikipedia.org/wiki/Biochemical_systems_equation
    rates = N @ deterministic_propensities
    sum_stochastic = sum(stochastic_propensities)

    return rates, sum_stochastic

In [ ]:
n_species = 10
n_pathways = 40
rate_involvement = np.random.poisson(0.3, (n_species,n_pathways))
y_expanded = np.random.random(n_species+1)
k = np.random.random(n_pathways)
k_of_t = lambda t: k
hitting_point = 0
t=0
deterministic_mask = np.random.choice(a=[False, True], size=(n_pathways), p=[0.2, 0.8])
stochastic_mask = ~deterministic_mask

stoichiometry = np.random.poisson(0.3, (n_species,n_pathways))

In [ ]:
assert(np.allclose(dydt(t, y_expanded, k_of_t, stoichiometry, rate_involvement, deterministic_mask, stochastic_mask, hitting_point), jit_dydt(t, y_expanded, k_of_t, stoichiometry, rate_involvement, deterministic_mask, stochastic_mask, hitting_point)))

In [ ]:
%%timeit
dydt(t, y_expanded, k_of_t, stoichiometry, rate_involvement, deterministic_mask, stochastic_mask, hitting_point)

In [ ]:
%%timeit
jit_dydt(t, y_expanded, k_of_t, stoichiometry, rate_involvement, deterministic_mask, stochastic_mask, hitting_point)

# Whole thing

In [2]:
import hybrid

In [3]:
n_species = 10
n_pathways = 40
rate_involvement = np.random.poisson(0.3, (n_species,n_pathways))
y0 = np.random.random(n_species)*50
k = np.random.random(n_pathways)
k_of_t = lambda t: k
hitting_point = 0
stoichiometry = np.random.poisson(0.3, (n_species,n_pathways))
rng = np.random.default_rng()

hybrid.forward_time(y0, [0, 10.0], lambda p: hybrid.partition_by_threshold(p, 100), k_of_t, stoichiometry, rate_involvement, rng)

AssertionError: integration step failed